In [6]:
%run -i ../code/wavelets_pca_with_zhost_1100_average_precision.py

In [7]:
%run -i ../code/wavelets_pca_with_zhost_1100_roc_auc.py

In [8]:
%load_ext memory_profiler

In [9]:
%load_ext line_profiler

In [10]:
import pandas as pd
import numpy as np

import pywt as wt
import pickle

import pymc3 as pm
%matplotlib inline

import matplotlib.pyplot as plt
import theano

In [ ]:
FAZER 12 LIDTAS, UMA PRA CADA COISA, NO CASO
4 FILTROS
3 VARIAVEIS ( MU, STD E T)

In [12]:
%run gen_files_array.ipynb

In [13]:
%run functions.ipynb

In [14]:
def get_wavelets(file, keys, wavelet = 'sym2', mlev = 2):
    wav = wt.Wavelet(wavelet)
    
    xstar, mu, stds = GP_prep(file, keys)
    
    for filt in keys: 
        coeffs = [np.array(wt.swt(mu[filt], wav, level=mlev)).flatten()]

    return np.concatenate(coeffs)

In [18]:
file = files[0]
filters = ['desg' , 'desi' , 'desr' , 'desz']

In [19]:
files

['../../data/raw_data/DES_SN144755.DAT',
 '../../data/raw_data/DES_SN745098.DAT',
 '../../data/raw_data/DES_SN621869.DAT',
 '../../data/raw_data/DES_SN394747.DAT',
 '../../data/raw_data/DES_SN585075.DAT',
 '../../data/raw_data/DES_SN162693.DAT',
 '../../data/raw_data/DES_SN046527.DAT',
 '../../data/raw_data/DES_SN106192.DAT',
 '../../data/raw_data/DES_SN279785.DAT',
 '../../data/raw_data/DES_SN498211.DAT',
 '../../data/raw_data/DES_SN719361.DAT',
 '../../data/raw_data/DES_SN322681.DAT',
 '../../data/raw_data/DES_SN068714.DAT',
 '../../data/raw_data/DES_SN520775.DAT',
 '../../data/raw_data/DES_SN346506.DAT',
 '../../data/raw_data/DES_SN484515.DAT',
 '../../data/raw_data/DES_SN242544.DAT',
 '../../data/raw_data/DES_SN105640.DAT',
 '../../data/raw_data/DES_SN544495.DAT',
 '../../data/raw_data/DES_SN580930.DAT',
 '../../data/raw_data/DES_SN371010.DAT',
 '../../data/raw_data/DES_SN265554.DAT',
 '../../data/raw_data/DES_SN216959.DAT',
 '../../data/raw_data/DES_SN216918.DAT',
 '../../data/raw

In [ ]:
def GP_matern52(t,x,y,yerr):
    np.random.seed(9)
    with pm.Model() as model:
        eta = 500
        lengthscale = 30
        cov = eta* pm.gp.cov.Matern52(1, lengthscale)        

        gp = pm.gp.Marginal(cov_func=cov)

        y_ = gp.marginal_likelihood("y", X=x, y=y, noise=yerr)

        #mp = pm.find_MAP(include_transformed=True)

        mu, var = gp.predict(t,  diag=True)
        fit = pd.DataFrame({"t": t.flatten(),
                        "mu_total": mu,
                        "sd_total": np.sqrt(var)})
        return fit
    
def GP_prep(file, filters, clean_method = '', gp_method = 'm32'):
    
    data_dict, t = get_df(file)
    t = t.reshape(len(t),1)
        
    mus = {filters[0] : [], filters[1] : [], filters[2] : [], filters[3] : []}
    stds = {filters[0] : [], filters[1] : [], filters[2] : [], filters[3] : []}
    

    for band, dat in data_dict.items():
        #with error treatment std_dev
        x, y, yerr = get_xys(cleaning_df(data_dict[band], method = clean_method, clean_neg = True))
        x = x.reshape(len(x),1)
        
        if gp_method == 'm32':
            fit = GP_matern32(t,x,y,yerr)
        elif gp_method == 'm52':
            fit = GP_matern52(t,x,y,yerr)
        elif gp_method == 'exp':
            fit = GP_expquad(t,x,y,yerr)
        elif gp_method == 'rat':
            fit = GP_ratquad(t,x,y,yerr)
    
        stds[band] = fit['mu_total']
        mus[band] = fit['sd_total']
                
    return t,mus,stds

In [ ]:
def get_wavelets(file, keys, wavelet = 'sym2', mlev = 2):
    wav = wt.Wavelet(wavelet)
    
    xstar, mu, stds = GP_prep(file, keys)
    
    for filt in keys: 
        coeffs = [np.array(wt.swt(mu[filt], wav, level=mlev)).flatten()]

    return np.concatenate(coeffs)

In [22]:
data_dict, t = get_df(file)
t = t.reshape(len(t),1)
mus = {filters[0] : [], filters[1] : [], filters[2] : [], filters[3] : []}
stds = {filters[0] : [], filters[1] : [], filters[2] : [], filters[3] : []}
    

In [25]:
#with error treatment std_dev
x, y, yerr = get_xys(cleaning_df(data_dict['desg'], method = '', clean_neg = True))
x = x.reshape(len(x),1)

In [40]:
shared_data = theano.shared(t, borrow=True)

In [41]:
shared_data.set_value(x)

In [38]:
shared_data.get_value()

array(1)

In [ ]:
data_dict, t = get_df(file)
t = t.reshape(len(t),1)
    
mus = {filters[0] : [], filters[1] : [], filters[2] : [], filters[3] : []}
stds = {filters[0] : [], filters[1] : [], filters[2] : [], filters[3] : []}

In [ ]:
np.random.seed(9)

results = []

filters = ['desg' , 'desi' , 'desr' , 'desz']

shared_t = theano.shared(t, borrow = True)
shared_x = theano.shared(x, borrow = True)
shared_y = theano.shared(y, borrow = True)
shared_yerr = theano.shared(yerr, borrow = True)

with pm.Model() as model:
    eta = 500
    lengthscale = 30
    cov = eta* pm.gp.cov.Matern52(1, lengthscale)        

    gp = pm.gp.Marginal(cov_func=cov)

    y_ = gp.marginal_likelihood("y", X=shared_x, y=shared_y, noise=shared_yerr)

    #mp = pm.find_MAP(include_transformed=True)

for file in files:
    
    data_dict, t = get_df(file)
    t = t.reshape(len(t),1)        
    mus = {filters[0] : [], filters[1] : [], filters[2] : [], filters[3] : []}
    stds = {filters[0] : [], filters[1] : [], filters[2] : [], filters[3] : []}
    
    for band, dat in data_dict.items():
        
        x, y, yerr = get_xys(cleaning_df(data_dict[band], method = '', clean_neg = True))
        x = x.reshape(len(x),1)
    
        shared_t.set_value(t)
        shared_x.set_value(x)
        shared_y.set_value(y)
        shared_yerr.set_value(yerr)

        with pm.Model() as model:
            mu, var = gp.predict(shared_t,  diag=True)
            results.append([shared_t.get_value().flatten(), mu,np.sqrt(var)])
        ###VER SE E PRA COLOCAR O FIT (AGORA RESULTS) FORA DO MODEL OU DENTRO, ESTOU FAZENDO UMA LISTA DE 3 ELEMENTOS#!!!!!!!!!!

In [ ]:
np.random.seed(9)

results = []

filters = ['desg' , 'desi' , 'desr' , 'desz']

shared_t = theano.shared(t, borrow = True)
shared_x = theano.shared(x, borrow = True)
shared_y = theano.shared(y, borrow = True)
shared_yerr = theano.shared(yerr, borrow = True)

with pm.Model() as model:
    eta = 500
    lengthscale = 30
    cov = eta* pm.gp.cov.Matern52(1, lengthscale)        

    gp = pm.gp.Marginal(cov_func=cov)

    y_ = gp.marginal_likelihood("y", X=shared_x, y=shared_y, noise=shared_yerr)

    #mp = pm.find_MAP(include_transformed=True)

for file in files:
    
    data_dict, t = get_df(file)
    t = t.reshape(len(t),1)        
    mus = {filters[0] : [], filters[1] : [], filters[2] : [], filters[3] : []}
    stds = {filters[0] : [], filters[1] : [], filters[2] : [], filters[3] : []}
    
    for band, dat in data_dict.items():
        
        x, y, yerr = get_xys(cleaning_df(data_dict[band], method = '', clean_neg = True))
        x = x.reshape(len(x),1)
    
        shared_t.set_value(t)
        shared_x.set_value(x)
        shared_y.set_value(y)
        shared_yerr.set_value(yerr)

        with model:
            mu, var = gp.predict(shared_t,  diag=True)
            results.append([shared_t.get_value().flatten(), mu,np.sqrt(var)])
        ###VER SE E PRA COLOCAR O FIT (AGORA RESULTS) FORA DO MODEL OU DENTRO, ESTOU FAZENDO UMA LISTA DE 3 ELEMENTOS#!!!!!!!!!!

In [ ]:
len(results)

In [72]:
#results = []
results.append([shared_t.get_value().flatten(), mu,np.sqrt(var)])


In [60]:
shared_t.set_value(t)
shared_x.set_value(x)
shared_y.set_value(y)
shared_yerr.set_value(yerr)

In [56]:
fit = pd.DataFrame({"t": shared_t.get_value().flatten()})

In [62]:
fit

,t,mu_total,sd_total
0,0.000000,4.541510,3.999240
1,1.211293,5.541493,3.562977
2,2.422586,6.542687,3.170048
3,3.633879,7.524614,2.812356
4,4.845172,8.464829,2.483443
5,6.056465,9.339880,2.182695
6,7.267758,10.126247,1.919231
7,8.479051,10.801306,1.713953
8,9.690343,11.344327,1.595307
9,10.901636,11.737556,1.583065


In [51]:
np.sqrt(var)

array([3.99923994, 3.56297721, 3.17004837, 2.8123555 , 2.48344251,
       2.18269543, 1.9192311 , 1.71395256, 1.59530736, 1.58306529,
       1.66947984, 1.82306909, 2.00851056, 2.19845045, 2.37426613,
       2.52278737, 2.63355213, 2.69840548, 2.71230041, 2.67444176,
       2.58965918, 2.47001717, 2.3364754 , 2.21952111, 2.15577779,
       2.1768049 , 2.29317841, 2.48912439, 2.73400978, 2.99621079,
       3.25006564, 3.47727675, 3.66614809, 3.81039682, 3.90803245,
       3.96033085, 3.97082689, 3.94426828, 3.88554602, 3.79897089,
       3.68849504, 3.55830576, 3.41325729, 3.25894255, 3.10126211,
       2.94545726, 2.79495197, 2.65058456, 2.51015536, 2.36817719,
       2.21678295, 2.04848723, 1.85924502, 1.65112979, 1.43502437,
       1.23407191, 1.08682016, 1.040446  , 1.12048241, 1.3060168 ,
       1.55234563, 1.8197595 , 2.08065181, 2.31725612, 2.51907597,
       2.68130012, 2.80368918, 2.88944916, 2.94380877, 2.97220902,
       2.97824467, 2.96200453, 2.92042026, 2.84929666, 2.74555

In [50]:
mu

array([ 4.54150985,  5.54149257,  6.54268704,  7.52461388,  8.46482917,
        9.33987994, 10.12624732, 10.80130578, 11.34432695, 11.73755595,
       11.96729479, 12.02470732, 11.90630825, 11.61414674, 11.15602205,
       10.54585687,  9.80374022,  8.95485704,  8.02806963,  7.05459691,
        6.06677157,  5.09683832,  4.17575661,  3.33197156,  2.5901165 ,
        1.96960902,  1.48318324,  1.13631213,  0.92814346,  0.8526696 ,
        0.89975073,  1.05601092,  1.30563106,  1.63106072,  2.01366951,
        2.43435767,  2.8741446 ,  3.3147538 ,  3.7392076 ,  4.13230494,
        4.48073013,  4.77303596,  4.99967581,  5.15309583,  5.22789166,
        5.22104673,  5.1323065 ,  4.96439678,  4.72298985,  4.41669871,
        4.05691765,  3.6572735 ,  3.23300287,  2.80037539,  2.3761492 ,
        1.97704179,  1.61920002,  1.31766678,  1.08535223,  0.93067056,
        0.85606654,  0.85805743,  0.92833817,  1.05498299,  1.2235476 ,
        1.41809898,  1.62220087,  1.81988255,  1.99661776,  2.14

In [47]:
np.sqrt(var)

array([3.99923994, 3.56297721, 3.17004837, 2.8123555 , 2.48344251,
       2.18269543, 1.9192311 , 1.71395256, 1.59530736, 1.58306529,
       1.66947984, 1.82306909, 2.00851056, 2.19845045, 2.37426613,
       2.52278737, 2.63355213, 2.69840548, 2.71230041, 2.67444176,
       2.58965918, 2.47001717, 2.3364754 , 2.21952111, 2.15577779,
       2.1768049 , 2.29317841, 2.48912439, 2.73400978, 2.99621079,
       3.25006564, 3.47727675, 3.66614809, 3.81039682, 3.90803245,
       3.96033085, 3.97082689, 3.94426828, 3.88554602, 3.79897089,
       3.68849504, 3.55830576, 3.41325729, 3.25894255, 3.10126211,
       2.94545726, 2.79495197, 2.65058456, 2.51015536, 2.36817719,
       2.21678295, 2.04848723, 1.85924502, 1.65112979, 1.43502437,
       1.23407191, 1.08682016, 1.040446  , 1.12048241, 1.3060168 ,
       1.55234563, 1.8197595 , 2.08065181, 2.31725612, 2.51907597,
       2.68130012, 2.80368918, 2.88944916, 2.94380877, 2.97220902,
       2.97824467, 2.96200453, 2.92042026, 2.84929666, 2.74555

In [48]:
fit

NameError: name 'fit' is not defined

In [ ]:
%%mprun 

samples_wavelets = []
results = []
keys = ['desg' , 'desi' , 'desr' , 'desz']


results = []
for f in files[0:1100]:
    
    results.append(get_wavelets(f,filters))

/home/felipematheus/anaconda3/lib/python3.7/site-packages/theano/tensor/basic.py:6611: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  result[diagonal_slice] = x
/home/felipematheus/anaconda3/lib/python3.7/site-packages/theano/tensor/basic.py:6611: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  result[diagonal_slice] = x
/home/felipematheus/anaconda3/lib/python3.7/site-packages/theano/tensor/basic.py:6611: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In th

In [33]:
%%mprun 

samples_wavelets = []
#results = []
keys = ['desg' , 'desi' , 'desr' , 'desz']


#results = []
for f in files[5:15]:
    
    results.append(get_wavelets(file,filters))

In [34]:
results

[array([ 3.11593732e+01,  4.20506050e+01,  4.84045383e+01,  4.84809021e+01,
         4.06576514e+01,  3.24769179e+01,  2.42520775e+01,  1.66240868e+01,
         1.05521184e+01,  7.17914935e+00,  6.65153368e+00,  8.34358282e+00,
         1.12505087e+01,  1.41623935e+01,  1.63127612e+01,  1.72969647e+01,
         1.69773628e+01,  1.56344494e+01,  1.38618978e+01,  1.24355249e+01,
         1.19504083e+01,  1.23739960e+01,  1.30883923e+01,  1.31249508e+01,
         1.19817555e+01,  9.93151134e+00,  7.79170046e+00,  6.74615381e+00,
         7.14118380e+00,  8.21643682e+00,  8.94619165e+00,  8.70053622e+00,
         7.47400459e+00,  5.98601723e+00,  5.09890343e+00,  4.86893950e+00,
         5.18519545e+00,  5.96538002e+00,  6.89077761e+00,  7.61160741e+00,
         7.78696697e+00,  7.15579109e+00,  5.92084296e+00,  4.78393679e+00,
         4.23490480e+00,  4.30347498e+00,  4.61843705e+00,  4.68919526e+00,
         4.48370953e+00,  4.40880621e+00,  4.79678345e+00,  5.54331313e+00,
         6.1

In [36]:
%lprun 

samples_wavelets = []
#results = []
keys = ['desg' , 'desi' , 'desr' , 'desz']


#results = []
for f in files[15:20]:
    
    results.append(get_wavelets(f,filters))

In [39]:
%mprun 

samples_wavelets = []
#results = []
keys = ['desg' , 'desi' , 'desr' , 'desz']


#results = []
for f in files[20:25]:
    
    results.append(get_wavelets(f,filters))

In [40]:
len(results)

25

In [41]:
%memit

samples_wavelets = []
#results = []
keys = ['desg' , 'desi' , 'desr' , 'desz']


#results = []
for f in files[25:30]:
    
    results.append(get_wavelets(f,filters))

peak memory: 823.14 MiB, increment: 0.20 MiB


In [42]:
%memit

samples_wavelets = []
#results = []
keys = ['desg' , 'desi' , 'desr' , 'desz']


#results = []
for f in files[30:40]:
    
    results.append(get_wavelets(f,filters))

peak memory: 945.99 MiB, increment: 0.00 MiB


In [43]:
%memit

samples_wavelets = []
#results = []
keys = ['desg' , 'desi' , 'desr' , 'desz']


#results = []
for f in files[40:50]:
    
    results.append(get_wavelets(f,filters))

peak memory: 1192.54 MiB, increment: 0.00 MiB


In [44]:
%memit

samples_wavelets = []
#results = []
keys = ['desg' , 'desi' , 'desr' , 'desz']


#results = []
for f in files[50:55]:
    
    results.append(get_wavelets(f,filters))

peak memory: 1438.37 MiB, increment: 0.00 MiB


In [45]:
%lprun

samples_wavelets = []
#results = []
keys = ['desg' , 'desi' , 'desr' , 'desz']


#results = []
for f in files[55:60]:
    
    results.append(get_wavelets(f,filters))

In [46]:
%lprun

samples_wavelets = []
#results = []
keys = ['desg' , 'desi' , 'desr' , 'desz']


#results = []
for f in files[60:75]:
    
    results.append(get_wavelets(f,filters))

In [57]:
%time

samples_wavelets = []
#results = []
keys = ['desg' , 'desi' , 'desr' , 'desz']


#results = []
for f in files[75:100]:
    
    results.append(get_wavelets(f,filters))

CPU times: user 6 µs, sys: 0 ns, total: 6 µs
Wall time: 14.1 µs


In [58]:
samples_wavelets = []
#results = []
keys = ['desg' , 'desi' , 'desr' , 'desz']


#results = []
for f in files[100:110]:
    
    results.append(get_wavelets(f,filters))

In [60]:
len(results)

110

In [63]:
pickle_out = open("../../models/frags/gp0_110.pickle","wb")
pickle.dump(results, pickle_out)
pickle_out.close()

In [ ]:
np.random.seed(9)
resp = []
clean_method = ''
wavelet = 'sym2'
mlev = 2
wav = wt.Wavelet(wavelet)
filters = ['desg' , 'desi' , 'desr' , 'desz']

i = 0
start = time.time()

with pm.Model() as model:
    eta = pm.HalfCauchy("eta", beta=2, testval=2.0)
    lengthscale = pm.Gamma("lenght", alpha=4, beta=0.1)
    cov = 500*eta**2 * pm.gp.cov.Matern32(1, lengthscale)        

    gp = pm.gp.Marginal(cov_func=cov)

    #mp = pm.find_MAP(include_transformed=True)
    
    for file in files[0:50]:
        data_dict, t = get_df(file)
        t = t.reshape(len(t),1)
            
        mus = {filters[0] : [], filters[1] : [], filters[2] : [], filters[3] : []}
        stds = {filters[0] : [], filters[1] : [], filters[2] : [], filters[3] : []}
    
        for band, dat in data_dict.items():
            #with error treatment std_dev
            x, y, yerr = get_xys(cleaning_df(data_dict[band], method = clean_method, clean_neg = True))
            x = x.reshape(len(x),1)
            
            y_ = gp.marginal_likelihood(str(band)+str(i), X=x, y=y, noise=yerr)
            mu, var = gp.predict(t,  diag=True)
            fit = pd.DataFrame({"t": t.flatten(),
                            "mu_total": mu,
                            "sd_total": np.sqrt(var)})
            #stds[band] = fit['mu_total']
            mus[band] = fit['sd_total']
            
        for filt in filters: 
            coeffs = [np.array(wt.swt(mus[filt], wav, level=mlev)).flatten()]            
        resp.append(np.concatenate(coeffs)) 
        i = i + 1
end = time.time()
print("Time running: " , (end - start)) 

In [13]:
def GP_matern32(t,x,y,yerr):
    np.random.seed(9)
    with pm.Model() as model:
        eta = pm.HalfCauchy("eta", beta=2, testval=2.0)
        lengthscale = pm.Gamma("lenght", alpha=4, beta=0.1)
        cov = 500*eta**2 * pm.gp.cov.Matern32(1, lengthscale)        

        gp = pm.gp.Marginal(cov_func=cov)

        y_ = gp.marginal_likelihood("y", X=x, y=y, noise=yerr)

        #mp = pm.find_MAP(include_transformed=True)

        mu, var = gp.predict(t,  diag=True)
        fit = pd.DataFrame({"t": t.flatten(),
                        "mu_total": mu,
                        "sd_total": np.sqrt(var)})
        return fit

In [33]:
(38.44158220291138*len(files)/10)/3600

22.761687951034972

In [34]:
start = time.time()
resp = get_wavelets(file,filters)
end = time.time()
print("Time running: " , (end - start)) 

Time running:  4.361891984939575


In [175]:
.013378877507315742 * len(files)/10

28.518415294594234

In [176]:
len(files)/8

2664.5

In [178]:
#tempo do 0 ao 100
0.1784640689028634 * 60

10.707844134171804

In [55]:

%memit

len(results)

peak memory: 2053.00 MiB, increment: 0.00 MiB


75